In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Hello**



**In this kernel, I tried to predict blood sugar levels of people who had stroke. I used Bagged Trees Regression and XGBoost. Lastly I compared accuracy scores of both models.**


# Data Exploration

In [ ]:
stroke = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
data= stroke.copy()

In [ ]:
data.head()

In [ ]:
data.describe().T

In [ ]:
df = data.loc[data["stroke"]== 1]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.head()

# Data Visiualization

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [ ]:
sns.countplot(data.stroke)

In [ ]:
data["work_type"].value_counts().plot.barh();

In [ ]:
sns.boxplot(x = "smoking_status", y = "avg_glucose_level", hue = "stroke", data = data);

In [ ]:
sns.catplot(y = "avg_glucose_level", kind = "violin", data = data);

In [ ]:
sns.pairplot(data,hue="stroke")

In [ ]:

sns.pairplot(df, kind  ="reg");

In [ ]:

from sklearn.preprocessing import LabelEncoder

number = LabelEncoder()
df['gender'] = number.fit_transform(df['gender'].astype(str))
df['ever_married'] = number.fit_transform(df['ever_married'].astype(str))


In [ ]:
df.head()

In [ ]:
dms = pd.get_dummies(df[['work_type', 'Residence_type', 'smoking_status']])



In [ ]:
y = df["avg_glucose_level"]


In [ ]:
X_ = df.drop(['work_type', 'Residence_type', 'smoking_status'], axis=1).astype('float64')



In [ ]:
X = pd.concat([X_, dms],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=1994)

**Bagged Tree Regression**

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.datasets import make_regression

In [ ]:
bag_model = BaggingRegressor(bootstrap_features = True)
bag_model.fit(X_train, y_train)

In [ ]:
bag_model.n_estimators

In [ ]:
bag_model.estimators_

In [ ]:
bag_model.estimators_samples_

In [ ]:
bag_model.estimators_features_

In [ ]:
bag_model.estimators_[1]

**Prediction**

In [ ]:
y_pred1 = bag_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

accuracy1= np.sqrt(mean_squared_error(y_test, y_pred1))

In [ ]:
second_y_pred = bag_model.estimators_[1].fit(X_train, y_train).predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, second_y_pred))

In [ ]:
seven_y_pred = bag_model.estimators_[4].fit(X_train, y_train).predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, seven_y_pred))

**XGBoost**

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
DM_train = xgb.DMatrix(data = X_train, label = y_train)
DM_test = xgb.DMatrix(data = X_test, label = y_test)

In [ ]:
xgb_model = XGBRegressor().fit(X_train, y_train)


**Prediction**

In [ ]:
y_pred2 = xgb_model.predict(X_test)
accuracy2 = np.sqrt(mean_squared_error(y_test, y_pred2))

In [ ]:
xgb_model

In [ ]:
models = {"XGBoost Score :" : accuracy2,
           "Bagged Tree Score :" : accuracy1 }


In [ ]:
import pandas as pd
ModelComparision = pd.DataFrame.from_dict(models,orient='index')
ModelComparision

# **Conclusion**



**XGBoost has less MSE than Bagged Trees model  in terms of predicting people's blood sugar levels who had stroke therefore XGBoost is more successful.**
